# Introduction to Trodes data import into `nelpy` 

<div class="alert alert-info">
<b>What is Trodes and what is supported by nelpy? </b>
</div>
[Trodes](http://spikegadgets.com/software/trodes.html) is 
> an open source, cross-platform software suite for neuroscience data acquisition and experimental control. If you use this to record data, the real joke is yourself. Mattias advises against it.

Importing some data from Trodes post recording from the Trodes data-type saved .rec file directly into `nelpy` is supported within the trodes.py file under the io portion of `nelpy`. In particular the data from the .recs that can be imported into `nelpy` include the following:
- Wide-band (or subsampled) ephys waveforms
- Auxiliary channels (as of right now only digital inputs and outputs are importable)
- Spike waveform snippits as determined within Trodes

Support is currently being expanded to support:
- Video frames and timestamp coordination
- Other auxiliary channels such as analog inputs/outputs, accelerometer data
- Other stuff (exciting and ambiguous!)

<br><div class="alert alert-info">
<b>How does it work?</b>
</div>
The `nelpy` import code relies heavily on modified versions of exportLFP, exportdio, exportspikes, etc. (essentially the Export functionality provided in the Trodes C++ code but I have modified them to take in added parameters). As such, my modifications to the Trodes export functions have been provided under the [resources folder on the `nelpy` github](https://github.com/eackermann/nelpy/tree/feature/TrodesImport/resources). BUT we will move away from this and just have pure Python implementations soon!

In [1]:
import numpy as np
from scipy import interpolate
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.io
import sys

import nelpy as nel  # recommended import for nelpy
import nelpy.plotting as npl  # recommended import for the nelpy plotting library
import nelpy.io.trodes as neltro

%matplotlib inline
npl.setup()
npl.set_palette(npl.colors.cows)
npl.setup(font='Universalis ADF Cd Std', font_scale=2, palette=npl.colors.cows, style='white',\
         rc=({'figure.figsize': (12, 6), 'font.size': 18, 'axes.labelsize': 25, \
             'axes.titlesize':33, 'legend.fontsize': 20, 'ytick.labelsize': 20, 'xtick.labelsize': 20}))

# Importing Wide-Band LFP data

## Get tetrodes and channels

In [2]:
tetrodes, channels = neltro.load_tetrode_channel_nums("/media/shayok/3TBData/Data/JChuTru/Install/Experiments/unthethered/06-30-2017/install_06-30-2017_00_09_56_merge_sd11.rec",\
                                 verbose = True)

Tetrodes:  [ 1  1  1  1  2  2  2  2  3  3  3  3  4  4  4  4  5  5  5  5  6  6  6  6  7
  7  7  7  8  8  8  8  9  9  9  9 10 10 10 10 11 11 11 11 12 12 12 12 13 13
 13 13 14 14 14 14 15 15 15 15 16 16 16 16 17 17 17 17 18 18 18 18 19 19 19
 19 20 20 20 20]
Channels:  [1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1
 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2
 3 4 1 2 3 4]


<br><div class="alert alert-info">
Above is not necessary unless you're loading up everything. On the other hand, if we wanted to disable particular channels from being loaded (e.g. I don't care about Tetrode 10 because it's in a different area of the brain that I'm using as reference or I don't care about tetrode 11 because it's lost, I can disable it like below), that is when this function comes handy as in the example below. 
</div>

In [3]:
tetrodes, channels = neltro.load_tetrode_channel_nums("/media/shayok/3TBData/Data/JChuTru/Install/Experiments/unthethered/06-30-2017/install_06-30-2017_00_09_56_merge_sd11.rec",\
                                                      disable_tetrodes = [10,11],verbose = True)

Disabling Tetrode 10 
Disabling Tetrode 11 
Tetrodes:  [ 1  1  1  1  2  2  2  2  3  3  3  3  4  4  4  4  5  5  5  5  6  6  6  6  7
  7  7  7  8  8  8  8  9  9  9  9 12 12 12 12 13 13 13 13 14 14 14 14 15 15
 15 15 16 16 16 16 17 17 17 17 18 18 18 18 19 19 19 19 20 20 20 20]
Channels:  [1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1
 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4]


<br><div class="alert alert-info">
Also, we can disable specific channels that we don't want. E.g. say tetrode 3 accidentally has one of it's channels messed up for some reason so we can disable particular channels from being loaded up. In the example below we remove tetrode 3 channel 2 and tetrode 4 channel 1.
</div>

In [4]:
tetrodes, channels = neltro.load_tetrode_channel_nums("/media/shayok/3TBData/Data/JChuTru/Install/Experiments/unthethered/06-30-2017/install_06-30-2017_00_09_56_merge_sd11.rec",\
                                                      disable_tetrodes = [3,4], disable_channels=[2,1],\
                                                      verbose = True)

Disabling Tetrode 3 | Channel(s) [2]
Disabling Tetrode 4 | Channel(s) [1]
Tetrodes:  [ 1  1  1  1  2  2  2  2  3  3  3  4  4  4  5  5  5  5  6  6  6  6  7  7  7
  7  8  8  8  8  9  9  9  9 10 10 10 10 11 11 11 11 12 12 12 12 13 13 13 13
 14 14 14 14 15 15 15 15 16 16 16 16 17 17 17 17 18 18 18 18 19 19 19 19 20
 20 20 20]
Channels:  [1 2 3 4 1 2 3 4 1 3 4 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3
 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4
 1 2 3 4]


<br><div class="alert alert-success">
In conclusion, this function here should be used as a helper if you're trying to load up a large subset of the tetrodes and channels or else it's just easier typing them in as inputs to the tetrode and channel keyword arguments in loading functions shown in the subsection below.
</div>

## Begin imports into AnalogSignalArrays

<br><div class="alert alert-info">
Quick example of how to load up all channels from all tetrodes from a recording session.
</div>

In [2]:
tetrodes, _ = neltro.load_tetrode_channel_nums("/home/shayok/Documents/Code/RippleDetectionAnalysis/TingData/Kant_stimulation_two_channel14_5_27.rec")
asa = neltro.load_wideband_lfp_rec("/home/shayok/Documents/Code/RippleDetectionAnalysis/TingData/Kant_stimulation_two_channel14_5_27.rec",\
                      "/home/shayok/Documents/Code/trodes/", tetrode = tetrodes, everything = True, verbose = True)
asa

/home/shayok/Documents/Code/trodes/bin/exportLFP -rec "/home/shayok/Documents/Code/RippleDetectionAnalysis/TingData/Kant_stimulation_two_channel14_5_27.rec" -userefs "0" -everything "1"
*****************Loading LFP Timestamps*****************
b'<Start settings>\n'
b'Description: LFP timestamps\n'
b'Byte_order: little endian\n'
b'Original_file: Kant_stimulation_two_channel14_5_27.rec\n'
b'Clock rate: 30000\n'
b'Decimation: 1\n'
b'Time_offset: 0\n'
b'Fields: <time uint32>\n'
Current file position 210
Done
*****************Loading LFP Data*****************
b'<Start settings>\n'
b'Description: LFP data for one channel\n'
b'Byte_order: little endian\n'
b'Original_file: Kant_stimulation_two_channel14_5_27.rec\n'
b'nTrode_ID: 1\n'
b'nTrode_channel: 1\n'
b'Clock rate: 30000\n'
b'Voltage_scaling: 0.195\n'
b'Decimation: 1\n'
b'First_timestamp: 4146660\n'
b'Reference: off\n'
b'Low_pass_filter: none\n'
b'Fields: <voltage int16>\n'
Current file position 323
Done
*****************Loading LFP Data***

/home/shayok/Documents/Code/nelpy/nelpy/core/_analogsignalarray.py:267: UserWarning: creating support from time and sampling rate, fs!


*****************Loading LFP Timestamps*****************
b'<Start settings>\n'
b'Description: LFP timestamps\n'
b'Byte_order: little endian\n'
b'Original_file: Kant_stimulation_two_channel14_5_27.rec\n'
b'Clock rate: 30000\n'
b'Decimation: 1\n'
b'Time_offset: 0\n'
b'Fields: <time uint32>\n'
Current file position 210
Done
*****************Loading LFP Data*****************
b'<Start settings>\n'
b'Description: LFP data for one channel\n'
b'Byte_order: little endian\n'
b'Original_file: Kant_stimulation_two_channel14_5_27.rec\n'
b'nTrode_ID: 2\n'
b'nTrode_channel: 1\n'
b'Clock rate: 30000\n'
b'Voltage_scaling: 0.195\n'
b'Decimation: 1\n'
b'First_timestamp: 4146660\n'
b'Reference: off\n'
b'Low_pass_filter: none\n'
b'Fields: <voltage int16>\n'
Current file position 323
Done
*****************Loading LFP Data*****************
b'<Start settings>\n'
b'Description: LFP data for one channel\n'
b'Byte_order: little endian\n'
b'Original_file: Kant_stimulation_two_channel14_5_27.rec\n'
b'nTrode_ID: 2\

[<AnalogSignalArray at 0x7fd2dc507080: 4 signals> for a total of 1:21:996 minutes,
 <AnalogSignalArray at 0x7fd2dc5071d0: 4 signals> for a total of 1:21:996 minutes,
 <AnalogSignalArray at 0x7fd2dc507160: 4 signals> for a total of 1:21:996 minutes,
 <AnalogSignalArray at 0x7fd2dc507278: 4 signals> for a total of 1:21:996 minutes,
 <AnalogSignalArray at 0x7fd2dc507358: 4 signals> for a total of 1:21:996 minutes,
 <AnalogSignalArray at 0x7fd2dc507438: 4 signals> for a total of 1:21:996 minutes,
 <AnalogSignalArray at 0x7fd2dc5074e0: 4 signals> for a total of 1:21:996 minutes,
 <AnalogSignalArray at 0x7fd2dc507588: 4 signals> for a total of 1:21:996 minutes]

In [6]:
[signal.labels for signal in asa]

[array(['t1c1', 't1c2', 't1c3', 't1c4'], 
       dtype='<U4'), array(['t2c1', 't2c2', 't2c3', 't2c4'], 
       dtype='<U4'), array(['t3c1', 't3c2', 't3c3', 't3c4'], 
       dtype='<U4'), array(['t4c1', 't4c2', 't4c3', 't4c4'], 
       dtype='<U4'), array(['t5c1', 't5c2', 't5c3', 't5c4'], 
       dtype='<U4'), array(['t6c1', 't6c2', 't6c3', 't6c4'], 
       dtype='<U4'), array(['t7c1', 't7c2', 't7c3', 't7c4'], 
       dtype='<U4'), array(['t8c1', 't8c2', 't8c3', 't8c4'], 
       dtype='<U4')]

In [ ]:
#plot!

<br><div class="alert alert-warning">
All data loaded up. It is worth noting here that when the everything flag is set to True, data is returned as a Python list of AnalogSignalArrays. In the example below, only specific channels are requested not "everything" and that results in a singular AnalogSignalArray being returned consisting of signals from the specific channels that are requested. Once again, the signals will be labeled so they are discriminatable.
</div>

<br><div class="alert alert-info">
Quick example of how to load up particular channels from a .rec file. 
</div>

<div class="alert alert-success">
This pretty much sums up how to load up lfp data into .rec files for the most general usage for most people. There are two more examples below that are useful for those loading up large amounts of lfp data into nelpy from .rec files and for those doing real-time analysis using Trodes as a part  of a closed loop system (this example is after the DIO section).
</div>

<br><div class="alert alert-info">
Saving data that doesn't fit into your RAM...Personal recommendation is to save it in ASAs if possible and then only analyze chunks of it.<br><br>

In this example, we are going to load up all data from all tetrodes of a 20 tetrode drive....well a couple tetrodes aren't useful so we don't care about them and they will be disabled.
</div>

In [3]:
import gc #garbage collectors are of lower priority to run so they need to be explicitly triggered
          #or we'll run out of RAM
def saveData(tetrode):
    """all channels for tetrode loaded up as ASA and use nelpy results container to pickle!
    """
    asa = neltro.load_wideband_lfp_rec("/media/shayok/3TBData/Data/JChuTru/Install/Experiments/unthethered/06-30-2017/install_06-30-2017_00_09_56_merge_sd11.rec",\
                                              "/home/shayok/Documents/Code/trodes/", tetrode = tetrode, everything = True,data_already_extracted=True)
    res = nel.ResultsContainer(asa = asa).save_pkl("/media/shayok/3TBData/Data/JChuTru/Install/Experiments/unthethered/06-30-2017/pickledData/t{}.pkl".format(tetrode),zip=False)
    print("Tetrode {} pickled!!".format(tetrode))
    gc.collect() #remove useless stuff prior to exiting function

In [4]:
tetrodes, _ = neltro.load_tetrode_channel_nums("/media/shayok/3TBData/Data/JChuTru/Install/Experiments/unthethered/06-30-2017/install_06-30-2017_00_09_56_merge_sd11.rec",
                                               disable_tetrodes=[2,7,8,18])
tetrodes = np.unique(tetrodes) #only need each tetrode once....should be clear why we need to do this from above subsection examples
for i in range(len(tetrodes)): #loop through and pickle all channels for each tetrode.
    saveData(tetrodes[i]) #doing this within a function so it's easier to remove stuff from RAM with the garbage collector

Disabling Tetrode 2 
Disabling Tetrode 7 
Disabling Tetrode 8 
Disabling Tetrode 18 


/home/shayok/Documents/Code/nelpy/nelpy/core/_analogsignalarray.py:267: UserWarning: creating support from time and sampling rate, fs!


File "/media/shayok/3TBData/Data/JChuTru/Install/Experiments/unthethered/06-30-2017/pickledData/t1.pkl" already exists! Aborting...
Tetrode 1 pickled!!
File "/media/shayok/3TBData/Data/JChuTru/Install/Experiments/unthethered/06-30-2017/pickledData/t3.pkl" already exists! Aborting...
Tetrode 3 pickled!!
File "/media/shayok/3TBData/Data/JChuTru/Install/Experiments/unthethered/06-30-2017/pickledData/t4.pkl" already exists! Aborting...
Tetrode 4 pickled!!
File "/media/shayok/3TBData/Data/JChuTru/Install/Experiments/unthethered/06-30-2017/pickledData/t5.pkl" already exists! Aborting...
Tetrode 5 pickled!!
Tetrode 6 pickled!!
Tetrode 9 pickled!!
Tetrode 10 pickled!!
Tetrode 11 pickled!!
Tetrode 12 pickled!!
Tetrode 13 pickled!!
Tetrode 14 pickled!!
Tetrode 15 pickled!!
Tetrode 16 pickled!!
Tetrode 17 pickled!!
Tetrode 19 pickled!!
Tetrode 20 pickled!!


<br><div class="alert alert-success">
Very simply put, this is all there is on how to load up LFP data from Trodes .rec files into nelpy. For further information check the function calls and flags. See AnalogSignalArrayDemo (plotting.ipynb might also be a good example notebook to view but plots are included in the ASA demo notebook) for how to manipulate the data inside and what all you can do! Also, if you're planning to use the nelpy framework EpochArrayDemo is a must see/understand!
</div>

# Importing Digital Input Channels

## Get channels

In [ ]:
channels = neltro.load_digital_channel_nums("/home/shayok/Desktop/testt.rec", disable_digital_channels=[3,8],verbose = True)

## Begin imports into numpy arrays <br>
<div class="alert alert-warning">
Note: This will be changed to EventArrays in the future (ETA: Unknown!)
</div>

<div class="alert alert-success">
Digital inputs have now been loaded into numpy arrays and can be used alongside LFP data.
</div>

# Importing Spike Data into `Nelpy`

# Importing Data for Real-Time Analysis

<div class="alert alert-info">
A few things must be done differently for real-time analysis of Trodes data. My particular experience doing this involves logging real-time LFP event detection to one of the Digital Input pins on the SpikeGadgets ECU and the OpenEphys FPGA. As such, I will go through how I did that just incase it is helpful to anyone else! <br><br>

As previously stated, exporting Trodes .rec files relies heavily on the modified versions of C++ export functions provided in the Trodes repository (under resources in the nelpy github repo). In general, if you're decimating (or subsampling) your data, it is recommended to use the subsample function provided within AnalogSignalArray class. Additionally, if you are filtering your data, it is recommended that you use the nelpy filter functions. However, for real-time analysis, we need to get virtually identical data to what modules communicating to Trodes would receive. For example, consider the [Ripple Detection Module](https://bitbucket.org/mkarlsso/trodes/src/ea50f3e9a633?at=rippleDetectionBeagleBoneStimModule). This module receives LFP data from Trodes which is (1) lowpass filtered at whatever frequency is specified within the config file (400 Hz if the KemereLab or SDModule config files are used) and (2) decimated by a factor of 1. Trodes itself uses an IIR filter to get LFP band signal to the modules and then "simple" decimation is done by sending every 10th sample to the Ripple Detection Module.
</div>

## Importing data with flags for Trodes filtering and decimation

## Importing digital input data for log of event detection

## Importing data from .rec file with `nelpy` filtering and subsample. <br>
<div class="alert alert-info">
This data will be used to compare phase shifts caused by the real-time filters. So this way, we will be able to compare when events transpired versus when they were detected and delays introduced by real-time analysis (computational + hardware delays).
</div>

## Example plots of data received and analyzed by real-time module vs data as it was logged by the data acquisition unit

<div class="alert alert-success">
Simple enough, right? If not contact (Shayok)[http://www.github.com/shayokdutta] he wrote this tutorial and should probably know what he's doing...
</div>